# Loading the vector database

In [1]:
import os
from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings

In [10]:
try:
    PROJECT_ROOT = os.path.dirname(os.path.abspath(__file__))
except:
    print("This should not be running on a notebook.")
    PROJECT_ROOT = os.path.abspath('.')
    
DATA_PATH = os.path.join(PROJECT_ROOT, "database_s3", "queplan_insurance")
DB_PATH = os.path.join(PROJECT_ROOT, "database_chromadb", "db")

This should not be running on a notebook.


In [11]:
vectorstore = Chroma(
    persist_directory=DB_PATH,
    embedding_function=GPT4AllEmbeddings()
)

# First retriever

In [12]:
retriever = vectorstore.as_retriever()

In [13]:
docs = retriever.aget_relevant_documents("Que contrato me recomiendas")

In [15]:
await docs

[Document(page_content='obligaciones del contrato, salvo aquellas que por su naturaleza deben ser cumplidos por el asegurado. Las', metadata={'page': 18, 'source': 'c:\\Users\\Agustin\\Proyectos_2024\\AnyoneAI-TPF\\backend\\database_s3\\queplan_insurance\\POL320200071.pdf'}),
 Document(page_content='4° Notificar al Asegurador, tan pronto sea posible una vez tomado conocimiento, de la ocurrencia de\ncualquier hecho que pueda constituir o constituya un siniestro, y\n \n5° Acreditar la ocurrencia del siniestro denunciado, y declarar fielmente y sin reticencia, sus circunstancias y\nconsecuencias.\n \nSi el Tomador o Contratante del seguro y el Asegurado son personas distintas, corresponde al Tomador o\nContratante el cumplimiento de las obligaciones del contrato, salvo aquellas que por su naturaleza deben ser\ncumplidas por el Asegurado. Las obligaciones del Tomador o Contratante podrán ser cumplidas por el\nAsegurado.', metadata={'page': 12, 'source': 'c:\\Users\\Agustin\\Proyectos_2024\

# First Retrieval Chain

In [17]:
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain.llms import OpenAI

In [19]:
qa_chain = RetrievalQA.from_chain_type(
    chain_type="stuff",
    retriever=retriever,
    llm=OpenAI(),
    return_source_documents=True,
    verbose=True
)

In [20]:
def process_llm_response(response):
    print(response['result'])
    print('\n\nSources:')
    [print(source.metadata['source']) for source in response['source_documents']]
    print("\n\n")

In [22]:
msg = 'Que contrato me recomiendas'
response = qa_chain.invoke(msg)



> Entering new RetrievalQA chain...

> Finished chain.


In [23]:
response

{'query': 'Que contrato me recomiendas',
 'result': ' No puedo recomendar un contrato en particular ya que soy un programa de ordenador y no tengo conocimiento de las necesidades o preferencias específicas de cada individuo. Es importante que consultes con un experto o investigues detenidamente antes de decidir qué contrato es el mejor para ti.',
 'source_documents': [Document(page_content='obligaciones del contrato, salvo aquellas que por su naturaleza deben ser cumplidos por el asegurado. Las', metadata={'page': 18, 'source': 'c:\\Users\\Agustin\\Proyectos_2024\\AnyoneAI-TPF\\backend\\database_s3\\queplan_insurance\\POL320200071.pdf'}),
  Document(page_content='4° Notificar al Asegurador, tan pronto sea posible una vez tomado conocimiento, de la ocurrencia de\ncualquier hecho que pueda constituir o constituya un siniestro, y\n \n5° Acreditar la ocurrencia del siniestro denunciado, y declarar fielmente y sin reticencia, sus circunstancias y\nconsecuencias.\n \nSi el Tomador o Contrata

In [24]:
process_llm_response(response)

 No puedo recomendar un contrato en particular ya que soy un programa de ordenador y no tengo conocimiento de las necesidades o preferencias específicas de cada individuo. Es importante que consultes con un experto o investigues detenidamente antes de decidir qué contrato es el mejor para ti.


Sources:
c:\Users\Agustin\Proyectos_2024\AnyoneAI-TPF\backend\database_s3\queplan_insurance\POL320200071.pdf
c:\Users\Agustin\Proyectos_2024\AnyoneAI-TPF\backend\database_s3\queplan_insurance\POL320180100.pdf
c:\Users\Agustin\Proyectos_2024\AnyoneAI-TPF\backend\database_s3\queplan_insurance\POL320150503.pdf
c:\Users\Agustin\Proyectos_2024\AnyoneAI-TPF\backend\database_s3\queplan_insurance\POL320200214.pdf





In [25]:
msg = 'Necesito una receta de mayonesa'
response = qa_chain.ainvoke(msg)

In [26]:
result = await response
result



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Necesito una receta de mayonesa',
 'result': ' No conozco una receta de mayonesa.',
 'source_documents': [Document(page_content='que ella y sus coberturas se encuentren vigentes a esa fecha y que haya transcurrido el período de carencia\nseñalado en las Condiciones Particulares.', metadata={'page': 1, 'source': 'c:\\Users\\Agustin\\Proyectos_2024\\AnyoneAI-TPF\\backend\\database_s3\\queplan_insurance\\POL320130223.pdf'}),
  Document(page_content='que ella y sus coberturas se encuentren vigentes a esa fecha y que haya transcurrido el período de carencia\nseñalado en las Condiciones Particulares.', metadata={'page': 16, 'source': 'c:\\Users\\Agustin\\Proyectos_2024\\AnyoneAI-TPF\\backend\\database_s3\\queplan_insurance\\POL320190074.pdf'}),
  Document(page_content='cuyo reembolso se solicita, salvo que se indique otro plazo en las Condiciones Particulares de la póliza. Con\ntodo, en caso de no renovación de la póliza el plazo de aviso de siniestro no podrá ser superior a trein

# Inspeccion de los parametros del Retriever

In [27]:
qa_chain.retriever.search_type

'similarity'

In [ ]:
qa_chain.retriever.search_type = "semantic"

In [28]:
qa_chain.retriever.vectorstore

In [29]:
# System Prompt and Human Prompt:

print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


# Second Retrieval Chain

In [31]:
from langchain_openai import ChatOpenAI


turbo_llm = ChatOpenAI(
    temperature=0.1777,
    model_name='gpt-3.5-turbo'
)

qa_chain_turbo = RetrievalQA.from_chain_type(
    chain_type="stuff",
    retriever=retriever,
    llm=turbo_llm,
    return_source_documents=True,
    verbose=True
)

In [32]:
msg = 'Requiero recomendaciones para un ARTICULO: "cobertura"'
llm_response = qa_chain_turbo.invoke(msg)



> Entering new RetrievalQA chain...

> Finished chain.


In [33]:
process_llm_response(llm_response)

Para redactar un artículo sobre "cobertura" en el contexto de seguros, podrías considerar los siguientes puntos:

1. **Definición de Cobertura**: Explica qué se entiende por cobertura en el ámbito de los seguros. Puedes mencionar que se refiere a las garantías y protecciones que ofrece una póliza de seguro en caso de determinados eventos.

2. **Tipos de Cobertura**: Describe los diferentes tipos de cobertura que suelen ofrecer las pólizas de seguros, como cobertura médica, cobertura de responsabilidad civil, cobertura de daños materiales, entre otros.

3. **Importancia de la Cobertura**: Destaca la importancia de contar con una cobertura adecuada en un seguro, ya que brinda tranquilidad y protección financiera en situaciones imprevistas.

4. **Periodo de Cobertura**: Menciona que la cobertura de un seguro suele tener un periodo determinado, que puede variar según el tipo de póliza y las condiciones particulares del contrato.

5. **Exclusiones y Limitaciones**: Es importante mencionar q

In [36]:
# System Prompt
print(qa_chain_turbo.combine_documents_chain.llm_chain.prompt.messages[0].prompt.template)

Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}


In [37]:
# Human Prompt
print(qa_chain_turbo.combine_documents_chain.llm_chain.prompt.messages[1].prompt.template)

{question}
